# Activation Data Exploration
Extract and examine ENTSOE reserve price data from ZIP files


In [2]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from datetime import datetime


In [3]:
import requests
API_KEY = "3da11da4-cb1c-41ea-8f2e-e80e188b9e4b"
periodStart = "202509010000"
periodEnd = "202510010000"

import requests

url = ("https://web-api.tp.entsoe.eu/api?"
"documentType=A84&"
"controlArea_Domain=10YHU-MAVIR----U&"
"businessType=A96&"
f"periodStart={periodStart}&"
f"periodEnd={periodEnd}&"
f"securityToken={API_KEY}")

payload={}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)
print(response.text)


<?xml version="1.0" encoding="utf-8"?>
<Balancing_MarketDocument xmlns="urn:iec62325.351:tc57wg16:451-6:balancingdocument:4:1">
  <mRID>258d7e22ac4d4532b31fa3ebac0aaed1</mRID>
  <revisionNumber>1</revisionNumber>
  <type>A84</type>
  <process.processType>A16</process.processType>
  <sender_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</sender_MarketParticipant.mRID>
  <sender_MarketParticipant.marketRole.type>A32</sender_MarketParticipant.marketRole.type>
  <receiver_MarketParticipant.mRID codingScheme="A01">10X1001A1001A450</receiver_MarketParticipant.mRID>
  <receiver_MarketParticipant.marketRole.type>A33</receiver_MarketParticipant.marketRole.type>
  <createdDateTime>2025-10-21T09:39:56Z</createdDateTime>
  <area_Domain.mRID codingScheme="A01">10YHU-MAVIR----U</area_Domain.mRID>
  <period.timeInterval>
    <start>2025-09-01T00:00Z</start>
    <end>2025-10-01T00:00Z</end>
  </period.timeInterval>
              <TimeSeries>
                <mRID>1</mRID>
                <

In [4]:
# Initialize zf and xml_files from `response` whether ZIP or plain XML
import io, zipfile
from pathlib import Path


content = response.content
try:
    # Try to open as ZIP
    zf = zipfile.ZipFile(io.BytesIO(content))
    xml_files = zf.namelist()
except zipfile.BadZipFile:
    # Wrap plain XML into a single-file ZIP so downstream code works unchanged
    buf = io.BytesIO()
    with zipfile.ZipFile(buf, 'w', compression=zipfile.ZIP_DEFLATED) as zout:
        zout.writestr('response.xml', content)
    buf.seek(0)
    zf = zipfile.ZipFile(buf, 'r')
    xml_files = zf.namelist()

print(f"Ready: {len(xml_files)} XML file(s)")


Ready: 1 XML file(s)


In [5]:
# Extract XML files into a single DataFrame (store raw XML text)
import re


rows = []

for xml_file in xml_files:
    print(f"\nProcessing {xml_file}...")

    # Read XML content and decode (handle BOM safely)
    xml_bytes = zf.read(xml_file)
    xml_text = xml_bytes.decode('utf-8-sig', errors='replace')

    # Split concatenated XML documents while preserving the declaration
    docs = [d for d in re.split(r'(?=<\?xml\s)', xml_text) if d.strip()]
    if not docs:
        docs = [xml_text]

    for i, doc in enumerate(docs):
        # Ensure declaration at start; if missing, add a minimal one
        cleaned = doc.lstrip()
        if not cleaned.startswith('<?xml'):
            cleaned = '<?xml version="1.0" encoding="UTF-8"?>\n' + cleaned

        name = f"{xml_file}_part_{i}" if len(docs) > 1 else xml_file
        rows.append({'file': name, 'xml': cleaned})

xml_docs_df = pd.DataFrame(rows)
print(f"Collected {len(xml_docs_df)} XML documents into xml_docs_df")
display(xml_docs_df.head())



Processing response.xml...
Collected 1 XML documents into xml_docs_df


,file,xml
0,response.xml,"<?xml version=""1.0"" encoding=""utf-8""?>\n<Balan..."


In [6]:
from lxml import etree as LET
print("Imported LET from lxml.etree")


Imported LET from lxml.etree


In [7]:
# Parse a single balancing XML into one DataFrame (metadata + all points)
from lxml import etree as LET
from pathlib import Path
import pandas as pd


def _lname(tag: str) -> str:
    return tag.split('}')[-1] if '}' in tag else tag


def parse_balancing_xml_to_df(xml_text: str) -> pd.DataFrame:
    root = LET.fromstring(xml_text.encode('utf-8'))

    # Document-level metadata (leaf nodes outside Point)
    meta = {}
    for el in root.iter():
        ln = _lname(el.tag)
        if ln == 'Point':
            continue
        # skip if any Point descendant
        if el.xpath('.//*[local-name()="Point"]'):
            continue
        if len(list(el)) == 0:
            val = (el.text or '').strip()
            if val:
                key = f"doc.{ln}"
                # keep first occurrence
                if key not in meta:
                    meta[key] = val
    # include root attributes
    for k, v in root.attrib.items():
        meta[f"doc.@{k}"] = v

    rows = []
    # Iterate TimeSeries → Period → Point
    for ts in root.xpath('.//*[local-name()="TimeSeries"]'):
        ts_meta = {}
        for child in ts:
            ln = _lname(child.tag)
            if ln in ('Period', 'timeInterval'):
                continue
            # capture simple leaf text under TS (one level)
            if len(list(child)) == 0:
                ts_meta[f"ts.{ln}"] = (child.text or '').strip()
        for per in ts.xpath('.//*[local-name()="Period"]'):
            # period context
            start = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="start"]/text()')
            end = per.xpath('.//*[local-name()="timeInterval"]/*[local-name()="end"]/text()')
            res = per.xpath('.//*[local-name()="resolution"]/text()')
            per_meta = {
                'per.start': start[0] if start else None,
                'per.end': end[0] if end else None,
                'per.resolution': res[0] if res else None,
            }
            for pt in per.xpath('.//*[local-name()="Point"]'):
                row = {}
                # Point fields (flatten one level deep)
                for ch in list(pt):
                    ln = _lname(ch.tag)
                    if len(list(ch)) == 0:
                        row[f"pt.{ln}"] = (ch.text or '').strip()
                    else:
                        # flatten nested elements under Point one level
                        for sub in ch:
                            row[f"pt.{ln}.{_lname(sub.tag)}"] = (sub.text or '').strip()
                # Merge: document meta + TS meta + period meta + point
                full = {**meta, **ts_meta, **per_meta, **row}
                rows.append(full)
    return pd.DataFrame(rows)

# Source XML: prefer first entry in zf/xml_files, else fallback to example file
if 'zf' in globals() and 'xml_files' in globals() and xml_files:
    xml_text_src = zf.read(xml_files[0]).decode('utf-8-sig', errors='replace')
else:
    example_path = Path('notebooks/ActivationPriceXML.txt')
    if not example_path.exists():
        example_path = Path('../notebooks/ActivationPriceXML.txt')
    xml_text_src = example_path.read_text(encoding='utf-8')

points_full_df = parse_balancing_xml_to_df(xml_text_src)
print(points_full_df.shape)
display(points_full_df.head())


(5340, 35)


,doc.mRID,doc.revisionNumber,doc.type,doc.process.processType,doc.sender_MarketParticipant.mRID,doc.sender_MarketParticipant.marketRole.type,doc.receiver_MarketParticipant.mRID,doc.receiver_MarketParticipant.marketRole.type,doc.createdDateTime,doc.area_Domain.mRID,...,ts.flowDirection.direction,ts.currency_Unit.name,ts.price_Measure_Unit.name,ts.curveType,per.start,per.end,per.resolution,pt.position,pt.activation_Price.amount,pt.imbalance_Price.category
0,258d7e22ac4d4532b31fa3ebac0aaed1,1,A84,A16,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-10-21T09:39:56Z,10YHU-MAVIR----U,...,A01,HUF,MWH,A03,2025-09-01T00:00Z,2025-10-01T00:00Z,PT15M,1,104248.33,A06
1,258d7e22ac4d4532b31fa3ebac0aaed1,1,A84,A16,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-10-21T09:39:56Z,10YHU-MAVIR----U,...,A01,HUF,MWH,A03,2025-09-01T00:00Z,2025-10-01T00:00Z,PT15M,2,0,A06
2,258d7e22ac4d4532b31fa3ebac0aaed1,1,A84,A16,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-10-21T09:39:56Z,10YHU-MAVIR----U,...,A01,HUF,MWH,A03,2025-09-01T00:00Z,2025-10-01T00:00Z,PT15M,4,0,A06
3,258d7e22ac4d4532b31fa3ebac0aaed1,1,A84,A16,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-10-21T09:39:56Z,10YHU-MAVIR----U,...,A01,HUF,MWH,A03,2025-09-01T00:00Z,2025-10-01T00:00Z,PT15M,5,0,A06
4,258d7e22ac4d4532b31fa3ebac0aaed1,1,A84,A16,10X1001A1001A450,A32,10X1001A1001A450,A33,2025-10-21T09:39:56Z,10YHU-MAVIR----U,...,A01,HUF,MWH,A03,2025-09-01T00:00Z,2025-10-01T00:00Z,PT15M,6,0,A06


In [8]:
points_full_df.columns

Index(['doc.mRID', 'doc.revisionNumber', 'doc.type', 'doc.process.processType',
       'doc.sender_MarketParticipant.mRID',
       'doc.sender_MarketParticipant.marketRole.type',
       'doc.receiver_MarketParticipant.mRID',
       'doc.receiver_MarketParticipant.marketRole.type', 'doc.createdDateTime',
       'doc.area_Domain.mRID', 'doc.start', 'doc.end', 'doc.businessType',
       'doc.mktPSRType.psrType', 'doc.flowDirection.direction',
       'doc.currency_Unit.name', 'doc.price_Measure_Unit.name',
       'doc.curveType', 'doc.resolution', 'doc.position',
       'doc.activation_Price.amount', 'doc.imbalance_Price.category',
       'ts.mRID', 'ts.businessType', 'ts.mktPSRType.psrType',
       'ts.flowDirection.direction', 'ts.currency_Unit.name',
       'ts.price_Measure_Unit.name', 'ts.curveType', 'per.start', 'per.end',
       'per.resolution', 'pt.position', 'pt.activation_Price.amount',
       'pt.imbalance_Price.category'],
      dtype='object')

In [9]:
points_full_df = points_full_df[
    ["per.start",
     "per.end", 
     "pt.position",
     "per.resolution",
     "doc.process.processType",
     "ts.flowDirection.direction",
     "ts.mktPSRType.psrType",
     "pt.activation_Price.amount"     ]]

In [10]:
points_full_df

,per.start,per.end,pt.position,per.resolution,doc.process.processType,ts.flowDirection.direction,ts.mktPSRType.psrType,pt.activation_Price.amount
0,2025-09-01T00:00Z,2025-10-01T00:00Z,1,PT15M,A16,A01,A03,104248.33
1,2025-09-01T00:00Z,2025-10-01T00:00Z,2,PT15M,A16,A01,A03,0
2,2025-09-01T00:00Z,2025-10-01T00:00Z,4,PT15M,A16,A01,A03,0
3,2025-09-01T00:00Z,2025-10-01T00:00Z,5,PT15M,A16,A01,A03,0
4,2025-09-01T00:00Z,2025-10-01T00:00Z,6,PT15M,A16,A01,A03,0
...,...,...,...,...,...,...,...,...
5335,2025-09-01T00:00Z,2025-10-01T00:00Z,2873,PT15M,A16,A02,A03,0
5336,2025-09-01T00:00Z,2025-10-01T00:00Z,2874,PT15M,A16,A02,A03,0
5337,2025-09-01T00:00Z,2025-10-01T00:00Z,2877,PT15M,A16,A02,A03,11970.386269
5338,2025-09-01T00:00Z,2025-10-01T00:00Z,2878,PT15M,A16,A02,A03,8067.757721


In [11]:
unique_counts = points_full_df.nunique(dropna=True).sort_values(ascending=False).to_frame("n_unique")
print(f"DataFrame shape: {unique_counts.shape}")
display(unique_counts)


DataFrame shape: (8, 1)


,n_unique
pt.position,2670
pt.activation_Price.amount,2592
ts.flowDirection.direction,2
per.start,1
per.resolution,1
per.end,1
doc.process.processType,1
ts.mktPSRType.psrType,1


In [12]:
import numpy as np
import pandas as pd

# Use local start if already computed; otherwise parse as-is (no tz conversion)
start = points_full_df.get('per.start_dt', pd.to_datetime(points_full_df['per.start'], errors='coerce'))

# Parse ISO 8601 resolution PT15M/PT30M/PT60M/PT1H -> minutes
res_ex = points_full_df['per.resolution'].str.upper().str.extract(r'PT(\d+)([HM])')
mins = pd.to_numeric(res_ex[0], errors='coerce')
mins = np.where(res_ex[1].eq('H'), mins * 60, mins)

# Positions are 1-based
pos = pd.to_numeric(points_full_df.get('pt.position', points_full_df['pt.position']), errors='coerce')
offset_min = (pos - 1) * mins

points_full_df['time_dt'] = start + pd.to_timedelta(offset_min, unit='m')
points_full_df['time_end_dt'] = points_full_df['time_dt'] + pd.to_timedelta(mins, unit='m')

points_full_df[['per.start','per.resolution','pt.position','time_dt','time_end_dt']].head()

,per.start,per.resolution,pt.position,time_dt,time_end_dt
0,2025-09-01T00:00Z,PT15M,1,2025-09-01 00:00:00+00:00,2025-09-01 00:15:00+00:00
1,2025-09-01T00:00Z,PT15M,2,2025-09-01 00:15:00+00:00,2025-09-01 00:30:00+00:00
2,2025-09-01T00:00Z,PT15M,4,2025-09-01 00:45:00+00:00,2025-09-01 01:00:00+00:00
3,2025-09-01T00:00Z,PT15M,5,2025-09-01 01:00:00+00:00,2025-09-01 01:15:00+00:00
4,2025-09-01T00:00Z,PT15M,6,2025-09-01 01:15:00+00:00,2025-09-01 01:30:00+00:00


In [13]:
points_full_df["per.start"] = pd.to_datetime(points_full_df["per.start"], errors='coerce')
points_full_df["per.end"] = pd.to_datetime(points_full_df["per.end"], errors='coerce')

In [14]:
points_full_df.groupby("ts.flowDirection.direction").count()

,per.start,per.end,pt.position,per.resolution,doc.process.processType,ts.mktPSRType.psrType,pt.activation_Price.amount,time_dt,time_end_dt
ts.flowDirection.direction,,,,,,,,,
A01,2670,2670,2670,2670,2670,2670,2670,2670,2670
A02,2670,2670,2670,2670,2670,2670,2670,2670,2670
